In [1]:
import seaborn as sns
import matplotlib.pyplot as plt

from data_load import *
from model_fit_functions import *

from sklearn.utils import shuffle

import pickle
import time


from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import SparsePCA

### Data load

In [2]:
data_dict =  load_split_pool(1346, 430, 10, scale = True)

Directory: Complete_Spectral_Data\Training_Data Physical properties shape: (1346, 5) Spectral prop shape: (1346, 110001)
Directory: Complete_Spectral_Data\Test_Data Physical properties shape: (810, 5) Spectral prop shape: (810, 110001)
Spectral data shape (2156, 110001)
Physical data shape (2156, 5)


### Definitions and transformations

In [10]:
def calculate_sparse_pca(alpha, n_components, train_df_pooled, val_df_pooled, sparse_pca_results):
    '''
    Estimate spars ePCA model, project the original features to the subspace and then recover the 
    original dat (as much as possible) and calculate RMSE loss.
    
    Inputs:
    - alpha: regularisation parameter
    - train_df_pooled: train set
    - val_df_pooled: validation set
    - sparse_pca_results: dictionary in whic hto store results
    
    Outputs:
    - dictionary storing results. Results include sparse pca object, transformed train and validation data '''
    
    print("Train shape", train_df_pooled.shape)
    
    n_samples = train_df_pooled.shape[0]
    total_var = np.trace( train_df_pooled.cov() )
    
    sparse_pca_results[alpha] = dict()
    
    # fit sparse pca
    transformer = SparsePCA(n_components = n_components, random_state=3455, n_jobs = -1, alpha = alpha)
    transformer.fit(train_df_pooled)

    # project dataset onto a subspace (i.e. reduce dimensionality)
    x_train_pca = transformer.transform(train_df_pooled)
    x_val_pca = transformer.transform(val_df_pooled)
    
    
    # Calculate explaend vairance ratio using QR decomposition (this is not used later on)
    # Since principal components are not orthogonal explained variance is based on successive OLS models rather
    # than covariance matrix eigenvalues.
    q, r = np.linalg.qr(x_train_pca / np.sqrt(n_samples - 1) ) 
    
    sparse_pca_results[alpha]["explained_var"] = np.diag(r**2) / total_var
    sparsity = (np.abs( transformer.components_ ).sum(axis = 0) > 1e-5).sum(axis = 0)
    
    sparse_pca_results[alpha]["total_var"] = total_var
    sparse_pca_results[alpha]["sparsity"] = sparsity

    # project back to the original space
    x_train_original_space = x_train_pca @ transformer.components_ # project back 
    x_val_original_space = x_val_pca @ transformer.components_ # project back
    # calculate RMSE
    loss = ((val_df_pooled - x_val_original_space) ** 2).sum().sum()/n_samples
    
    # store the results in a dictionary which is then returned
    sparse_pca_results[alpha]["spca_object"] = transformer
    sparse_pca_results[alpha]["x_train"] = x_train_pca 
    sparse_pca_results[alpha]["x_val"] = x_val_pca
    sparse_pca_results[alpha]["components"] = transformer.components_

    sparse_pca_results[alpha]["x_train_original_space"] = x_train_original_space
    sparse_pca_results[alpha]["x_val_original_space"] = x_val_original_space
    
    sparse_pca_results[alpha]["loss"] = loss
    print(loss)

    dump_object("obj_" + str(alpha) + "_" + str(n_components), sparse_pca_results)


    return( sparse_pca_results )

In [141]:
spca_results = dict()


start_time = time.time()

spca_results = dict()

spca_results = calculate_sparse_pca(5, 250, train_df_pooled, val_df_pooled, spca_results)

print("--- %s seconds ---" % (time.time() - start_time))


Train shape (1346, 36667)
27822.23872040349
--- 12106.216319799423 seconds ---


In [ ]:
start_time = time.time()

spca_results = dict()

spca_results = calculate_sparse_pca(2.5, 250, train_df_pooled, val_df_pooled, spca_results)

print("--- %s seconds ---" % (time.time() - start_time))

Train shape (1346, 36667)


In [ ]:
start_time = time.time()

spca_results = dict()

spca_results = calculate_sparse_pca(0.1, 150, train_df_pooled, val_df_pooled, spca_results)

print("--- %s seconds ---" % (time.time() - start_time))